**Data Fetch**

In [1]:
import pandas as pd
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import time
import csv
#from tweepy import twitter_credentials

In [2]:
consumer_key="ruNYgRO4DnQZzZnxEWy6KoOCB"
consumer_secret="uc3kJgjj51bYXvEBobUoxzuhKc5InioIJPCQ8jEn6mmisJEtcO"
access_key="4673892211-uXEjUOfqZoWSqfYEWhSQpclwdURzZRTO2LaAd6N"
access_secret="Wq2uTSMNiJ9JJ40dbmPsnBZsKpDD1eFCRGV9Xp0eNZCaN"

In [3]:
#Function to extract tweets:
def get_tweets(username):
    # Authorization to consumer key and consumer secret
    auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
    #Access to user's access key and access secret
    auth.set_access_token(access_key, access_secret)
    #calling api
    api=tweepy.API(auth)
    #3200 tweets to be extracted
    number_of_tweets=3200
    tweets=api.user_timeline(screen_name=username)
    #Empty array
    tmp=[]
    ##Create array of tweet information: username,tweet id, date/time, text
    tweets_for_csv=[tweet.text for tweet in tweets]   #csv file created
    for j in tweets_for_csv:
        #Appending tweets to the empty array tmp
        tmp.append(j)
    print(tmp)

In [4]:
if __name__ =='__main__':
    get_tweets("@1DonalMurphy")

["The ground is drying out nicely at Roscommon so it's unlikely to suit any soft ground specialists. A few each-way s… https://t.co/aC7emJcsdy", 'Gorgeous evening in The Rower, Co Kilkenny. Little poser 😍. https://t.co/H5F6dz4M8x', "@irishracing @WexfordRacecour @LizDoyleRacing Both horses and jockeys were okay. We wouldn't publish it otherwise.", 'That break again 😍😍.\nHope Pestana goes all the way.\n#BoyleSportsIGD https://t.co/5Gmc6XCcro', 'The sixth in Wexford this afternoon. Poetry in motion, bar a few out the back 😁 https://t.co/srpLZro7o4', 'Another afternoon at the beautiful Curragh. Check out my formscans by following the link \nhttps://t.co/eWEDSEoZkf https://t.co/1mbHI0G2wM', "@SineadCashin @RTERadio1 @RTEdoconone Never heard of that story. Thanks Sinead, I'll tune in.", "@Gavbrophy I do indeed. Bellewstown is probably the closest you'd get to being at a race meeting nowadays.", '@Gavbrophy You could go to Bellewstown tomorrow. There is ample space for viewing the races just

**Topic Modelling**

In [ ]:
#import some basic libararies
import numpy as np
import pandas as pd
from pandas import DataFrame
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#import data
tw=pd.read_excel(r'C:\Users\hp\Downloads\twitter.xlsx')
tw.head()

In [ ]:
import re                              
from textblob import TextBlob

In [ ]:
##Fetching important variable
tw=DataFrame(tw,columns=['user_id','created_at','screen_name','text','display_text_width'])  
tw.head(5)

In [ ]:
# Create a function to clean the tweets
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text)   ##Removed @mentions
    text=re.sub(r'#','',text)   ##Remove hashtags
    text=re.sub(r'RT[\s]+','',text)   ##Removing RT
    text=re.sub(r'https?\/\/\S+','',text)  ##Remove hyperlink
    text=re.sub(r"\'", "", text) 
    text=re.sub(r'\s+', ' ', text) 
    text=re.sub(r'\S*@\S*\s?', '', text) 
    return text

##Cleaning the text
tw['text']=tw['text'].apply(cleanTxt)
tw.head()

In [ ]:
#set some english stopwords
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
stopwords.add("http")
stopwords.add("https")
stopwords.add("amp")
stopwords.add("im")
stopwords.add("co")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer    #from sklearn module imported countvectorizer function
new_vect=CountVectorizer(ngram_range=(1,2),stop_words=stopwords)   ###Fixed the Vectorizer
new_vect    ##print vectorizer

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation      
doc_term_matrix=new_vect.fit_transform(tw['text'].values.astype('U'))    ###made the document-term-matrix

In [ ]:
## Applying LDA and fitting document term matrix in it.
LDA=LatentDirichletAllocation(n_components=10,random_state=0)  
LDA.fit(doc_term_matrix)                                                 

In [ ]:
## Calculated Ist and top topic words document term matrix in array format.
first_topic=LDA.components_[0]
print("first_topic:",first_topic)
top_topic_words=first_topic.argsort()[-10:]
print("top_topic_words:",top_topic_words)

In [ ]:
##Showing Top Topic Wrods from vectorizer.
for i in top_topic_words:
    print(new_vect.get_feature_names()[i])

In [ ]:
##Showing Top 10 Topics from tweets and in that 10 topics 10 top words. 
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([new_vect.get_feature_names()[i] 
           for i in topic.argsort()[-10:]])
    print('\n')

In [ ]:
#Assigning Topics to every texts i.e which topic number comes under which twitts.
topic_values=LDA.transform(doc_term_matrix)
topic_values.shape
tw['topic']=topic_values.argmax(axis=1)
tw=DataFrame(tw,columns=['user_id','created_at','screen_name','text','display_text_width','topic'])  ## Adding topic Variable into DataFrame
tw.head(5)